In [ ]:
import numpy as np

import cvxportfolio as cvx

from .universes import NDX100 as UNIVERSE

# times
START = '2016-01-01'
END = None # today

# Currently (~2024) shorting large cap US stocks costs about this,
# in annualized percentages
BORROW_FEES = 0.25

# We set the bid-ask spreads at 5 basis points
SPREAD = 5E-4

# This is the b multiplier of the (3/2) power term in TransactionCost
MARKET_IMPACT = 1.

policy = cvx.SinglePeriodOptimization(
    objective=cvx.ReturnsForecast()
        - cvx.Gamma() * cvx.FactorModelCovariance(num_factors=10)
        - cvx.Gamma() * cvx.TransactionCost(a=SPREAD/2, b=MARKET_IMPACT)
        - cvx.Gamma() * cvx.HoldingCost(short_fees=BORROW_FEES),
    constraints = [
        cvx.DollarNeutral(), cvx.MarketNeutral(), cvx.LeverageLimit(7)],
    # this solver is somewhat more robust than ECOS, but less efficient
    solver='CLARABEL',
    # this is a CVXPY compilation flag that disables a feature that is very
    # useful (cache a semi-compiled problem) but its implementation scales
    # badly with the problem size; if you increase number of factors or
    # universe size, you may have to uncomment the next line
    # ignore_dpp=True,
)

simulator = cvx.MarketSimulator(
    universe=UNIVERSE,
    costs = [
        cvx.TransactionCost(a=SPREAD/2, b=MARKET_IMPACT),
        cvx.HoldingCost(short_fees=BORROW_FEES)])

# automatic hyper-parameter optimization (by greedy grid search)
simulator.optimize_hyperparameters(
    policy, start_time=START, end_time=END,
    objective='sharpe_ratio')

print('Optimized policy hyper-parameters:')
print(policy)

# back-test the policy with optimized hyper-parameters
result = simulator.backtest(policy, start_time=START, end_time=END)

print("Optimized policy back-test result:")
print(result)

# plot
result_figure = result.plot()

# check that back-tested returns of the strategy are uncorrelated with the
# market benchmark
market_benchmark_returns = simulator.backtest(
    cvx.MarketBenchmark(), start_time=START, end_time=END).returns

print('Correlation of strategy returns with benchmark:')
print(np.corrcoef(result.returns, market_benchmark_returns)[0, 1])